In [1]:
# importamos las librerías que necesitamos

# Tratamiento de datos
import pandas as pd
import numpy as np

# Visualización
import matplotlib.pyplot as plt
import seaborn as sns

# Evaluar linealidad de las relaciones entre las variables
# y la distribución de las variables
import scipy.stats as stats
from scipy.stats import shapiro, poisson, chisquare, expon, kstest


# Configuración
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

# Gestión de los warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv("../files/rating-and-performance-nonull.csv" , index_col = 0)

1. Dadas las situaciones, decide si se trata de una distribución de tipo exponencial o de poisson. A nivel teórico, no hace falta que lo demuestres con código. 

- Imagina que estás registrando el tiempo que transcurre entre la llegada de clientes a una tienda en minutos. ¿Se asemeja más a una distribución exponencial o de Poisson?

**Respuesta**: Si estamos registrando el tiempo transcurrido entre llegada y llegada de clientes a una tienda, sería más parecido a una distribución exponencial, ya que esta describe el tiempo entre eventos sucesivos.

- Si observas el número de accidentes que ocurren en una intersección en un día determinado, ¿tiene más sentido pensar en una distribución exponencial o de Poisson?

**Respuesta**: Si estamos interesados en observar el número de accidentes quer ocurren en la intersección y en un día determinado (y no el tiempo que hay entre accidente y accidente), entonces tendría más sentido emplear la distribución de Poisson. Esta describe la probabilidad de que un número específico de eventos ocurran en un intervalo de tiempo dado.

   - Cuando las personas esperan en una cola para recibir atención al cliente en una tienda, se registra el tiempo que esperan en minutos. ¿Parece seguir una distribución exponencial o de Poisson?

**Respuesta**: Se seguiría una distribución exponencial, según el razonamiento expresado previamente.

  - Si supervisas el número de llamadas telefónicas que llegan a un centro de atención al cliente en una hora, ¿sería más apropiado pensar en una distribución exponencial o de Poisson?

**Respuesta**: Sería más apropiado pensar en una distribución de Poisson, por los motivos indicados anteriormente.

   - Si mides el tiempo entre la llegada de correos electrónicos a tu bandeja de entrada durante el día, ¿se ajusta mejor a una distribución exponencial o de Poisson? 

**Respuesta**: Se ajustaría mejor una distribución exponencial, por los motivos descritos con anterioridad.

2. Ejercicios intervalo de confianza: Para cada uno de los ejercicios interpreta los resultados y piensa que ventaja a nivel de negocio podemos sacar de calcularlo. 

   - Trabajas para una plataforma de comercio en línea y deseas determinar el intervalo de confianza al 95% para el precio promedio de productos con una cantidad de media de valoraciones al producto de 500.

In [43]:
df_500 = df[(df['mean_product_ratings_count'] > 500)]

media_precio = df_500['mean_product_prices'].mean()

error_precio = stats.sem(df_500['mean_product_prices'])

confianza_precio = 0.95

libertad_precio = len(df_500['mean_product_prices']) - 1

valor_critico_precio = stats.t.ppf((1 + confianza_precio) / 2, df=libertad_precio)

limite_inferior_precio = media_precio - valor_critico_precio * error_precio
limite_superior_precio = media_precio + valor_critico_precio * error_precio

print("Intervalo de Confianza para el precio promedio de productos con una cantidad media de valoraciones de producto de 500:")
print(f"Media Muestral: {np.round(media_precio, 2)}")
print(f"Error Estándar: {np.round(error_precio, 2)}")
print(f"Nivel de Confianza: {confianza_precio}")
print(f"Valor Crítico: {np.round(valor_critico_precio, 2)}")
print(f"Intervalo de Confianza: ({np.round(limite_inferior_precio, 2)}, {np.round(limite_superior_precio, 2)})")

Intervalo de Confianza para el precio promedio de productos con una cantidad media de valoraciones de producto de 500:
Media Muestral: 8.29
Error Estándar: 0.12
Nivel de Confianza: 0.95
Valor Crítico: 1.96
Intervalo de Confianza: (8.04, 8.53)


**Respuesta**: Gracias a este cálculo, podríamos realizar una segmentación de mercado (es decir, comprender las tendencias de los consumidores y decidir estrategias de marketing específicas) y tomar decisiones de manera informada.

   - Eres un analista de ventas en una plataforma de comercio electrónico y quieres calcular el intervalo de confianza al 90% para el número promedio de unidades vendidas por producto.

In [44]:
media_unidades = df['total_units_sold'].mean()

error_unidades = stats.sem(df['mean_units_sold_per_product'])

confianza_unidades = 0.90

libertad_unidades = len(df['mean_units_sold_per_product']) - 1

valor_critico_unidades = stats.t.ppf((1 + confianza_unidades) / 2, df=libertad_unidades)

limite_inferior_unidades = media_unidades - valor_critico_unidades * error_unidades
limite_superior_unidades = media_unidades + valor_critico_unidades * error_unidades

print("Intervalo de Confianza para el promedio de unidades vendidas por producto:")
print(f"Media Muestral: {np.round(media_unidades, 2)}")
print(f"Error Estándar: {np.round(error_unidades, 2)}")
print(f"Nivel de Confianza: {confianza_unidades}")
print(f"Valor Crítico: {np.round(valor_critico_unidades, 2)}")
print(f"Intervalo de Confianza: ({np.round(limite_inferior_unidades, 2)}, {np.round(limite_superior_unidades, 2)})")

Intervalo de Confianza para el promedio de unidades vendidas por producto:
Media Muestral: 11299.4
Error Estándar: 202.91
Nivel de Confianza: 0.9
Valor Crítico: 1.65
Intervalo de Confianza: (10965.44, 11633.36)


**Respuesta**: Saber estos datos permite llevar a cabo una planificación de inventario (evitar escasez de los productos más populares), llevar a cabo una estimación de ingresos futuros o establecer nuevos objetivos y métricas de desempeño.

   - Trabajas en la gestión de inventario de una tienda en línea y necesitas calcular el intervalo de confianza al 99% para el precio promedio de productos según su país de origen.

In [45]:
grouped = df.groupby('origin_country')['mean_units_sold_per_product'].agg(['mean', 'std', 'count'])

grouped['standard_error'] = grouped['std'] / np.sqrt(grouped['count'])

confianza_paises = 0.99

libertad_paises = grouped['count'] - 1

valor_critico_paises = stats.t.ppf((1 + confianza_paises) / 2, df=libertad_paises)

grouped['margin_of_error'] = valor_critico_paises * grouped['standard_error']

grouped['lower_bound'] = grouped['mean'] - grouped['margin_of_error']
grouped['upper_bound'] = grouped['mean'] + grouped['margin_of_error']

print("Intervalo de Confianza al 99% para el precio promedio de productos según el país de origen:")
display(grouped[['lower_bound', 'mean', 'upper_bound']])

Intervalo de Confianza al 99% para el precio promedio de productos según el país de origen:


,lower_bound,mean,upper_bound
origin_country,,,
AT,NaN,100.000000,NaN
CN,3874.357806,4409.754077,4945.150348
GB,NaN,1000.000000,NaN
SG,10050.000000,10050.000000,10050.000000
US,342.275417,1405.161290,2468.047164
VE,13.611584,70.000000,126.388416


**Respuesta**: Segmentación de mercado (al entender las diferencias en los precios promedio entre países, puedes adaptar tus estrategias de marketing y segmentar tus campañas publicitarias para llegar de manera más efectiva a los clientes en cada país) y toma de decisiones de precios (conocer el intervalo de confianza del precio promedio por país de origen te permite establecer precios competitivos y rentables para tus productos en diferentes mercados)

   - Eres un gerente de ventas en una plataforma de comercio electrónico y quieres determinar el intervalo de confianza al 95% para la valoración promedio de los vendedores según si ofrecen envío express o no.

In [28]:
grouped2 = df.groupby('shipping_is_express')['rating_y'].agg(['mean', 'std', 'count'])

grouped2['standard_error'] = grouped2['std'] / np.sqrt(grouped2['count'])

confianza_ship = 0.95

libertad_ship = grouped2['count'] - 1

valor_critico_ship = stats.t.ppf((1 + confianza_ship) / 2, df=libertad_ship)

grouped2['margin_of_error'] = valor_critico_ship * grouped2['standard_error']

grouped2['lower_bound'] = grouped2['mean'] - grouped2['margin_of_error']
grouped2['upper_bound'] = grouped2['mean'] + grouped2['margin_of_error']

print("Intervalo de Confianza al 95% para el rating de vendedores según si ofrecen envío express o no:")
display(grouped2[['lower_bound', 'mean', 'upper_bound']])

Intervalo de Confianza al 95% para el rating de vendedores según si ofrecen envío express o no:


,lower_bound,mean,upper_bound
shipping_is_express,,,
No,4.021596,4.031728,4.041860
Yes,4.198233,4.282000,4.365767


**Respuesta**: Podemos evaluar el rendimiento del servicio de envío express y optimizar estrategias de envío.

   - Trabajas en el departamento de precios de una tienda en línea y necesitas calcular el intervalo de confianza al 90% para el descuento promedio de productos según la reputación del vendedor (alta, media o baja).

In [33]:
df['seller_reputation2'] = df['rating_y'].apply(lambda x : 'alta' if x > 4.5 else ('media' if x > 3.9 else 'baja')) 

grouped3 = df.groupby('seller_reputation2')['discount_percentage'].agg(['mean', 'std', 'count'])

grouped3['standard_error'] = grouped3['std'] / np.sqrt(grouped3['count'])

confianza_dis = 0.90

libertad_dis = grouped3['count'] - 1

valor_critico_dis = stats.t.ppf((1 + confianza_dis) / 2, df=libertad_dis)

grouped3['margin_of_error'] = valor_critico_dis * grouped3['standard_error']

grouped3['lower_bound'] = grouped3['mean'] - grouped3['margin_of_error']
grouped3['upper_bound'] = grouped3['mean'] + grouped3['margin_of_error']

print("Intervalo de Confianza al 90% para descuento promedio de productos según la reputación del vendedor:")
display(grouped3[['lower_bound', 'mean', 'upper_bound']])

Intervalo de Confianza al 90% para descuento promedio de productos según la reputación del vendedor:


,lower_bound,mean,upper_bound
seller_reputation2,,,
alta,-421.146835,-208.956000,3.234835
baja,-224.223717,-190.068487,-155.913258
media,-211.114240,-193.160892,-175.207544


**Respuesta**: Gracias a estos cálculos podemos evaluar el rendimiento de los descuentos y optimizar estrategias de precios.

3. Ejercicios prueba de hipótesis:

   - Prueba de Hipótesis sobre el precio promedio de productos con o sin empleados calificados

      - *Contexto:* Trabajas en una plataforma de comercio en línea que ofrece productos de moda. Quieres saber si hay una diferencia significativa en el precio promedio de productos entre aquellos vendidos por empleados calificados (rating_employee > 4) y aquellos vendidos por empleados menos calificados (rating_employee <= 4). Esto te ayudará a tomar decisiones sobre la estrategia de precios y capacitación de empleados.

      - *Hipótesis Nula (H0):* No hay diferencia significativa en el precio promedio de productos entre empleados con buena y mala reputación. 
      
      - *Hipótesis Alternativa (H1):* Existe una diferencia significativa en el precio promedio de productos entre empleados calificados y menos calificados.

In [34]:
# vamos a definir una función para poder hacer el t-test de Student

def prueba_hipotesis(*args):
    
    # lo primero que tenemos que hacer es mirar si las varianzas son iguales o no
    if len(args) == 2:
        p_valor_varianza = stats.levene(*args, center = "median")[1]
    else:
        p_valor_varianza = stats.bartlett(*args)[1]
    
    if p_valor_varianza > 0.05:
        # realizamos la prueba t de Student
        t_stat, p_valor = stats.ttest_ind(*args, equal_var=True)
    else:
        t_stat, p_valor = stats.ttest_ind(*args, equal_var=False)
        
    # Establecemos un nivel de significancia (alfa)
    alfa = 0.05

    # comparamos el p-valor con el nivel de significancia
    if p_valor < alfa:
        print("Rechazamos la hipótesis nula.")
        print("Hay una diferencia significativa entre los dos grupos.")
    else:
        print("No podemos rechazar la hipótesis nula.")
        print("No hay evidencia suficiente para afirmar una diferencia significativa entre los dos grupos.")

In [35]:
grupo_calificado = df[df['rating_y'] > 4]['price']
grupo_menos_calificado = df[df['rating_y'] <= 4]['price']

prueba_hipotesis(grupo_calificado, grupo_menos_calificado)

No podemos rechazar la hipótesis nula.
No hay evidencia suficiente para afirmar una diferencia significativa entre los dos grupos.


   - Prueba de Hipótesis sobre el número de unidades vendidas en envío express y envío estándar

      - *Contexto:* Eres un gerente de operaciones de envío en una plataforma de comercio electrónico. Quieres determinar si el número promedio de unidades vendidas difiere cuando los productos se envían con envío express (shipping_option_name = "Express Shipping") en comparación con envío estándar (shipping_option_name = "Livraison standard"). Esto te ayudará a tomar decisiones sobre la logística de envío.

      - *Hipótesis Nula (H0):* No hay diferencia significativa en el número promedio de unidades vendidas entre envío express y envío estándar.

      - *Hipótesis Alternativa (H1):* Existe una diferencia significativa en el número promedio de unidades vendidas entre envío express y envío estándar.

In [38]:
grupo_express = df[df['shipping_option_name'] == 'Livraison Express']['total_units_sold']
grupo_estandar = df[df['shipping_option_name'] == 'Livraison standard']['total_units_sold']

prueba_hipotesis(grupo_express, grupo_estandar)

No podemos rechazar la hipótesis nula.
No hay evidencia suficiente para afirmar una diferencia significativa entre los dos grupos.
